# Let's build an agentic search loop!

This notebook has a basic agentic search loop

* We have a set of furniture in our catalog
* We tell the Agent our preferences
* The agent uses the search tool to recommend furniture

In this notebook we mostly get a feel for how the overall loop works by unrolling it step by step

In [27]:
import os
os.environ["CHEAT_AT_SEARCH_DATA_PATH"] = "/home/jovyan/data"

from cheat_at_search.data_dir import mount
mount(use_gdrive=False)    # colab, share data across notebook runs on gdrive
# mount(use_gdrive=False) # <- colab without gdrive
# mount(use_gdrive=False, manual_path="/path/to/directory")  # <- force data path to specific directory, ie you're running locally.

## Get an OpenAI Key

This will prompt you for an OpenAI Key to interact with GPT-5

In [28]:
from cheat_at_search.data_dir import key_for_provider
from openai import OpenAI

OPENAI_KEY = key_for_provider("openai")

openai = OpenAI(api_key=OPENAI_KEY)

You're going to be prompted for your API key. This will be stored in a local file
If you'd prefer to set it as an environment variable, set it as:
    export OPENAI_API_KEY=your_api_key_here


Enter your openai_api_key:  ········


## Load the Wayfair corpus

We'll recommend products only from this corpus

In [29]:
from cheat_at_search.wands_data import corpus
corpus

,product_id,product_name,product_class,category hierarchy,product_description,product_features,rating_count,average_rating,review_count,features,doc_id,title,description,category,sub_category,cat_subcat,title_snowball,description_snowball
0,0,solid wood platform bed,Beds,Furniture / Bedroom Furniture / Beds & Headboa...,"good , deep sleep can be quite difficult to ha...",overallwidth-sidetoside:64.7|dsprimaryproducts...,15.0,4.5,15.0,"[overallwidth-sidetoside:64.7, dsprimaryproduc...",0,solid wood platform bed,"good , deep sleep can be quite difficult to ha...",Furniture,Bedroom Furniture,Furniture / Bedroom Furniture,"Terms({'bed', 'platform', 'wood', 'solid'})","Terms({'age', 'accessori', 's', 'this', 'bette..."
1,1,all-clad 7 qt . slow cooker,Slow Cookers,Kitchen & Tabletop / Small Kitchen Appliances ...,"create delicious slow-cooked meals , from tend...",capacityquarts:7|producttype : slow cooker|pro...,100.0,2.0,98.0,"[capacityquarts:7, producttype : slow cooker, ...",1,all-clad 7 qt . slow cooker,"create delicious slow-cooked meals , from tend...",Kitchen & Tabletop,Small Kitchen Appliances,Kitchen & Tabletop / Small Kitchen Appliances,"Terms({'cooker', 'qt', '7', 'slow', 'clad', 'a...","Terms({'everyth', 's', 'this', 'healthi', 'mea..."
2,2,all-clad electrics 6.5 qt . slow cooker,Slow Cookers,Kitchen & Tabletop / Small Kitchen Appliances ...,prepare home-cooked meals on any schedule with...,features : keep warm setting|capacityquarts:6....,208.0,3.0,181.0,"[features : keep warm setting, capacityquarts:...",2,all-clad electrics 6.5 qt . slow cooker,prepare home-cooked meals on any schedule with...,Kitchen & Tabletop,Small Kitchen Appliances,Kitchen & Tabletop / Small Kitchen Appliances,"Terms({'6', 'electr', '5', 'cooker', 'qt', 'sl...","Terms({'hour', 'safe', 'on', 'this', 'meal', '..."
3,3,all-clad all professional tools pizza cutter,"Slicers, Peelers And Graters",Browse By Brand / All-Clad,this original stainless tool was designed to c...,overallwidth-sidetoside:3.5|warrantylength : l...,69.0,4.5,42.0,"[overallwidth-sidetoside:3.5, warrantylength :...",3,all-clad all professional tools pizza cutter,this original stainless tool was designed to c...,Browse By Brand,All-Clad,Browse By Brand / All-Clad,"Terms({'tool', 'pizza', 'clad', 'all', 'cutter...","Terms({'pizza', 's', 'to', 'origin', 'through'..."
4,4,baldwin prestige alcott passage knob with roun...,Door Knobs,Home Improvement / Doors & Door Hardware / Doo...,the hardware has a rich heritage of delivering...,compatibledoorthickness:1.375 '' |countryofori...,70.0,5.0,42.0,"[compatibledoorthickness:1.375 '' , countryofo...",4,baldwin prestige alcott passage knob with roun...,the hardware has a rich heritage of delivering...,Home Improvement,Doors & Door Hardware,Home Improvement / Doors & Door Hardware,"Terms({'prestig', 'knob', 'round', 'baldwin', ...","Terms({'moment', 'entri', 'to', 'offer', 'on',..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
42989,42989,malibu pressure balanced diverter fixed shower...,Shower Panels,Home Improvement / Bathroom Remodel & Bathroom...,the malibu pressure balanced diverter fixed sh...,producttype : shower panel|spraypattern : rain...,3.0,4.5,2.0,"[producttype : shower panel, spraypattern : ra...",42989,malibu pressure balanced diverter fixed shower...,the malibu pressure balanced diverter fixed sh...,Home Improvement,Bathroom Remodel & Bathroom Fixtures,Home Improvement / Bathroom Remodel & Bathro...,"Terms({'panel', 'fix', 'shower', 'divert', 'ba...","Terms({'rain', 'panel', 'bodi', 'chrome', 'div..."
42990,42990,emmeline 5 piece breakfast dining set,Dining Table Sets,Furniture / Kitchen & Dining Furniture / Dinin...,,basematerialdetails : steel| : gray wood|ofhar...,1314.0,4.5,864.0,"[basematerialdetails : steel, : gray wood, of...",42990,emmeline 5 piece breakfast dining set,,Furniture,Kitchen & Dining Furniture,Furniture / Kitchen & Dining Furniture,"Terms({'set', 'dine', '5', 'piec', 'breakfast

### Index the furniture

We'll index title and description with basic stemming to be able to retrieve them

In [30]:
from searcharray import SearchArray
from cheat_at_search.tokenizers import snowball_tokenizer

corpus['title_snowball'] = SearchArray.index(corpus['title'].fillna(''), snowball_tokenizer)
corpus['description_snowball'] = SearchArray.index(corpus['description'].fillna(''), snowball_tokenizer)

2025-12-06 14:06:37,958 - searcharray.indexing - INFO - Indexing begins w/ 4 workers


INFO:searcharray.indexing:Indexing begins w/ 4 workers


2025-12-06 14:06:37,964 - searcharray.indexing - INFO - 0 Batch Start tokenization


INFO:searcharray.indexing:0 Batch Start tokenization


2025-12-06 14:06:37,970 - searcharray.indexing - INFO - Tokenizing 42994 documents


...

INFO:searcharray.indexing:Batch tokenization complete


2025-12-06 14:06:39,935 - searcharray.indexing - INFO - (main thread) Processing 1 batch results


INFO:searcharray.indexing:(main thread) Processing 1 batch results


2025-12-06 14:06:40,021 - searcharray.indexing - INFO - Indexing from tokenization complete


INFO:searcharray.indexing:Indexing from tokenization complete


## Create a furniture products search function

Here is a function that searches a Wayfair product dataset. It's just a Python function that returns top 10 pieces of furniture.

Right now we'll call it directly, soon we'll help ChatGPT interact with this.

In [31]:
import numpy as np
from typing import Union

def search_furniture(keywords: str) -> list[dict[str, Union[str, int, float]]]:
    """Search the available furniture products, get top 10 furniture.

    This is just a naive BM25 / keyword search of the product title and description.
    Don't expect sophisticated synonyms or semantic search. Just basic keyword with
    some stemming.

    """
    print("search", keywords)
    required_keywords = [term[1:] for term in keywords.split() if term.startswith("+")]
    bm25_scores = np.zeros(len(corpus))
    for term in snowball_tokenizer(keywords):
        bm25_scores += corpus['title_snowball'].array.score(term) * 7
        bm25_scores += corpus['description_snowball'].array.score(term) * 4

    for required_term in snowball_tokenizer(" ".join(required_keywords)):
        required_score = (corpus['title_snowball'].array.score(required_term) +
                          corpus['description_snowball'].array.score(required_term))
        bm25_scores[required_score == 0] = 0

    top_k_indices = np.argsort(bm25_scores)[-10:][::-1]
    bm25_scores = bm25_scores[top_k_indices]
    top_movies = corpus.iloc[top_k_indices].copy()
    top_movies.loc[:, 'score'] = bm25_scores

    results = []
    for id, row in top_movies.iterrows():
        results.append({
            'id': row['doc_id'],
            'title': row['title'],
            'description': row['description'],
            'score': row['score']
        })
    return results



search_furniture("geometric style +couch")

search geometric style +couch


[{'id': 1217,
  'title': 'extra large and wide couch riser',
  'description': 'our largest and oversized couch , furniture , and bed riser . made for those extra-large couch and furniture legs . we created these to allow one time stacking . tested to lift over 6,000 pounds - we made it heavy duty . includes a leather pad to keep legs from sliding off the top and a rubber base to prevent slipping on the floor . fits almost all sofas , couches , beds , large legs , and feet .',
  'score': 37.797197341918945},
 {'id': 25326,
  'title': 'pixar cars 2 in 1 flip open kids foam couch',
  'description': "now your little one can have their very own place to sit with the marshmallow furniture children 's 2-in-1 flip open foam kids sofa . this couch for toddlers is the perfect place for them to call their own while they read , eat snacks , watch tv , or nap . this marshmallow furniture children 's 2-in-1 flip open foam futon-style sofa is made of lightweight foam so kiddos can move it around from

## Describe the search tool to the LLM

There is a specific schema for telling OpenAI about our tools / functions. However, the cheat at search library has added some conveniences:

* We use the function name as the name to OpenAI
* We use the doc string to get a description
* The typing information gets encoded in parameters and return value

So IMPORTANTLY -- all these things are part of the prompt

### Annoying serialization / deserialization

When we get it in an OpenAI-friendly format, we also keep around some book-keeping for annoying serialization / deserialization of the arguments

With this we get some plumbing information in a 3-tuple
* The arguments to pass (as one pydantic struct)
* The tool as OpenAI sees it
* The function to call to delegate to this tool

Don't get too lost in the weeds here. In future notebooks, cheat-at-search helper code will just do this for you behind the scenes.

In [32]:
from cheat_at_search.agent.pydantize import make_tool_adapter
search_tool = make_tool_adapter(search_furniture)

tool_info = {search_furniture.__name__: search_tool}
tool_info

{'search_furniture': (cheat_at_search.agent.pydantize.Search_furnitureArgs,
  {'type': 'function',
   'name': 'search_furniture',
   'description': "Search the available furniture products, get top 10 furniture.\n\n    This is just a naive BM25 / keyword search of the product title and description.\n    Don't expect sophisticated synonyms or semantic search. Just basic keyword with\n    some stemming.",
   'parameters': {'properties': {'keywords': {'title': 'Keywords',
      'type': 'string'}},
    'required': ['keywords'],
    'title': 'Search_furnitureArgs',
    'type': 'object'}},
  <function cheat_at_search.agent.pydantize.make_tool_adapter.<locals>.call_from_tool(d: dict)>)}

In [33]:
import json
print(json.dumps(tool_info['search_furniture'][1], indent=2))

{
  "type": "function",
  "name": "search_furniture",
  "description": "Search the available furniture products, get top 10 furniture.\n\n    This is just a naive BM25 / keyword search of the product title and description.\n    Don't expect sophisticated synonyms or semantic search. Just basic keyword with\n    some stemming.",
  "parameters": {
    "properties": {
      "keywords": {
        "title": "Keywords",
        "type": "string"
      }
    },
    "required": [
      "keywords"
    ],
    "title": "Search_furnitureArgs",
    "type": "object"
  }
}


## Gather initial prompts

* System prompt - the general task, to lookup furniture in our catalog to recommend
* User prompt - what the user has given as a task (here listing the movies they like)



In [34]:
system_prompt = """
Users are coming to explore a catalog of furniture.

Use the search tool (search_furniture) to help them
"""

inputs = []
inputs.append({"role": "system", "content": system_prompt})

prompt = """
Help me find a modern couch with geometric style
"""

inputs.append({"role": "user", "content": prompt})
inputs

[{'role': 'system',
  'content': '\nUsers are coming to explore a catalog of furniture.\n\nUse the search tool (search_furniture) to help them\n'},
 {'role': 'user',
  'content': '\nHelp me find a modern couch with geometric style\n'}]

## Make a single call to the LLM (non-tool)

We make a call to the agent, and get some decent recommendations back. But without using the catalog to lookup what's available.

In [35]:
resp = openai.responses.create(
    model="gpt-5",
    input=inputs,
)
resp

Response(id='resp_0821c4360cbb6126006934387603b08192add8d0bae05e6c28', created_at=1765030006.0, error=None, incomplete_details=None, instructions=None, metadata={}, model='gpt-5-2025-08-07', object='response', output=[ResponseReasoningItem(id='rs_0821c4360cbb61260069343876e77881929e67a849f1d61166', summary=[], type='reasoning', content=None, encrypted_content=None, status=None), ResponseReasoningItem(id='rs_0821c4360cbb6126006934387bfc788192a7a1e0a38317f990', summary=[], type='reasoning', content=None, encrypted_content=None, status=None), ResponseOutputMessage(id='msg_0821c4360cbb612600693438871ca481928531f9351927beee', content=[ResponseOutputText(annotations=[], text='Great—happy to help. A couple quick preferences so I can run a precise search:\n- Size/configuration: standard 3-seat, sectional, or loveseat? Any max width?\n- Upholstery: fabric, performance fabric, leather, or velvet?\n- Color palette?\n- Budget range?\n- Any must-haves (sleeper, chaise, bench seat, metal legs, etc.)

## Tell the agent about a tool

We tell the agent about the tools it can use. Note the agent doesn't directly "call" the tools (there's no magic backdoor from OpenAI -> this notebook). Insetad, once the agent knows about the tool it can REQUEST we call them.

In [36]:
resp = openai.responses.create(
    model="gpt-5",
    input=inputs,
    tools=[tool[1] for tool in tool_info.values()],
)
inputs += resp.output
resp

Response(id='resp_010d6d11f16056aa006934388ad0fc81a1af9c609453b41e43', created_at=1765030026.0, error=None, incomplete_details=None, instructions=None, metadata={}, model='gpt-5-2025-08-07', object='response', output=[ResponseReasoningItem(id='rs_010d6d11f16056aa006934388b72ac81a19647f070c2a93d9b', summary=[], type='reasoning', content=None, encrypted_content=None, status=None), ResponseFunctionToolCall(arguments='{"keywords":"modern couch geometric"}', call_id='call_oba02EVrxBiLSMNiwkZz8cQP', name='search_furniture', type='function_call', id='fc_010d6d11f16056aa006934388c0d1c81a1a17db5c5c3b61074', status='completed')], parallel_tool_calls=True, temperature=1.0, tool_choice='auto', tools=[FunctionTool(name='search_furniture', parameters={'properties': {'keywords': {'title': 'Keywords', 'type': 'string'}}, 'required': ['keywords'], 'title': 'Search_furnitureArgs', 'type': 'object', 'additionalProperties': False}, strict=True, type='function', description="Search the available furniture 

In [37]:
resp.output

[ResponseReasoningItem(id='rs_010d6d11f16056aa006934388b72ac81a19647f070c2a93d9b', summary=[], type='reasoning', content=None, encrypted_content=None, status=None),
 ResponseFunctionToolCall(arguments='{"keywords":"modern couch geometric"}', call_id='call_oba02EVrxBiLSMNiwkZz8cQP', name='search_furniture', type='function_call', id='fc_010d6d11f16056aa006934388c0d1c81a1a17db5c5c3b61074', status='completed')]

## Issue the requested calls to your search tool

Now we do the magic of calling the tools directly

You can ignore the `annoying_tool_marshalling` its doing some lookups and plumbings to go between the JSON arguments and the Python world we have here.

The important thing is that we recieve a tool call request, we call the requested tool (here by doing a lookup and getting `tool_fn` that just wraps the search function)

Then we go on to append those all back into th inputs, with a JSON response, the call id, and a note to OpenAI this is a "function_call_output"

In [38]:
def annoying_tool_marshalling(item) -> dict:

    # Lookup how the agent wants to call the tool
    tool_name = item.name
    tool = tool_info[tool_name]
    ToolArgsModel = tool[0]
    tool_fn = tool[2]
    fn_args: ToolArgsModel = ToolArgsModel.model_validate_json(item.arguments)

    # The tool call function itself (ie search)
    # wrapped in something helping with serialization
    py_resp, json_resp = tool_fn(fn_args)

    # 4. Provide function call results to the model
    return {
        "type": "function_call_output",
        "call_id": item.call_id,
        "output": json_resp,
    }


for item in resp.output:
    if item.type == "function_call":
        tool_name = item.name

        # *** Get the tool, and package
        # up the call to the tool (our python function)
        tool_response = annoying_tool_marshalling(item)
        # 4. Provide function call results to the model
        inputs.append(tool_response)



search modern couch geometric


## Call LLM again with tool responses

Now we tell the LLM about the tool responses.

It may ask to search more (ie we should continue the loop). Or it may be done and have a final response


In [39]:
resp = openai.responses.create(
    model="gpt-5",
    input=inputs,
    tools=[tool[1] for tool in tool_info.values()],
)
inputs += resp.output

resp

Response(id='resp_010d6d11f16056aa006934388cfaec81a198c14d9df3873e7e', created_at=1765030029.0, error=None, incomplete_details=None, instructions=None, metadata={}, model='gpt-5-2025-08-07', object='response', output=[ResponseReasoningItem(id='rs_010d6d11f16056aa006934388e0f8c81a18107c55cd024beec', summary=[], type='reasoning', content=None, encrypted_content=None, status=None), ResponseFunctionToolCall(arguments='{"keywords":"geometric couch"}', call_id='call_gWXnWt4ykvSLRwQDegxbLB50', name='search_furniture', type='function_call', id='fc_010d6d11f16056aa0069343890f61c81a1a16b8c8b992e257a', status='completed'), ResponseFunctionToolCall(arguments='{"keywords":"geometric sofa"}', call_id='call_sBidWGzI7HW8lPjwinyUK16j', name='search_furniture', type='function_call', id='fc_010d6d11f16056aa0069343891158881a19f80d70b0b1b912c', status='completed'), ResponseFunctionToolCall(arguments='{"keywords":"modern sofa geometric"}', call_id='call_J2qpJSSJlFjtBfF73wkLERGt', name='search_furniture', ty

## Put it all in one loop

In [40]:
import textwrap

system_prompt = """
Users are coming to explore a catalog of furniture.

Use the search tool (search_furniture) to help them.

Use trial and error to figure out how best use the search tool.
"""

def agentic_search(query: str, summary=True) -> str:

    inputs = []
    inputs.append({"role": "system", "content": system_prompt})

    inputs.append({"role": "user", "content": query})


    tool_calls = True
    resp = None
    while tool_calls:
        resp = openai.responses.create(
            model="gpt-5",
            input=inputs,
            tools=[tool[1] for tool in tool_info.values()],
            reasoning={
                "effort": "medium",
                "summary": "auto" if summary else "none"
            }
        )
        inputs += resp.output
        if summary:
            for item in resp.output:
                if item.type == "reasoning":
                    print("Reasoning:")
                    for summary_item in item.summary:
                        print(textwrap.fill(summary_item.text, 80), "\n")
                    item.summary = []

        for item in resp.output:
            tool_calls = False
            if item.type == "function_call":
                tool_calls = True
                # *** Get the tool, and package
                # up the call to the tool (our python function)
                tool_response = annoying_tool_marshalling(item)

                # 4. Provide function call results to the model
                inputs.append(tool_response)
    return resp

resp = agentic_search("geometric sofa")

Reasoning:
**Searching for furniture**  I'm helping explore a furniture catalog, and the
user is looking for a "geometric sofa." I think the best step is to use the
search_furniture tool with that exact phrase. The tool seems to work by
searching titles and descriptions, so it’s straightforward to input "geometric
sofa" as the keywords. Let's proceed and see what options come up! 

search geometric sofa
Reasoning:
**Refining sofa search**  I'm noticing the search results aren't great; they
mostly included geometric pillows and rugs instead of sofas. There are a couple
of sofa items, like the "Dariana twin convertible sofa," which seems
misclassified. I'm not finding anything explicitly labeled as "geometric sofa."
I think I could refine the search terms to include phrases like "sofa geometric
pattern" or "modern geometric sofa." Also, I could run multiple searches in
parallel using the tool to improve results. Let's go ahead and do that! 

search geometric pattern sofa
search patterned

In [41]:
print(resp.output[-1].content[-1].text)

Got it—are you looking for:
- A sofa with geometric-patterned upholstery, or
- A modern/ angular (geometric) silhouette?

Also, what size do you prefer (sofa, loveseat, sectional), any color/material, and budget range?

Closest matches I can show you now:
- 33595 — Convertible Modular Sectional Sofa (U-shaped, reversible, ottomans). Clean, blocky geometry and modular layout.
- 654 — Modern L-Shaped Reversible Sectional Sofa. Minimal, angular lines with solid wood legs.
- 26718 — Grazielli 71" Armless Loveseat. Low-profile, sleek geometric silhouette with slanted legs.
- 41464 — Holguin Convertible Sofa. Simple modern lines; converts to a bed.

If you want geometric prints specifically, I can pair a solid sofa with geometric accessories (pillows/rug) like:
- 5998 — Noriega Turquoise Geometric Pillow
- 41803 — Ruby Rita Mid-Century Geometric Area Rug

Tell me your preferences and I’ll refine the picks.


## What other problems would users have to solve

* Movie by title
  * misspellings different phrasings, etc
  * with different
* Movie by genre
  *
* Movies based on movies I like
* Movies based on my mood
* Movies by the same directory
* Keyword search

##

## Next Steps

*